In [158]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import json
import random



In [331]:


def election_result_collection(data, json):

    planId = []
    democratsPercentage = []
    republicanPercentage = []
    total_plans = len(json.get("barData", []))
   
    
    for item in data:
        #total_district = len(item.get("ids",{}))
        total_district = json["totalDistricts"]
        elec_res = item.get("elec_res", {})

        totals_for_party = elec_res.get("percents_for_party", {})
        dem_totals = totals_for_party.get("Democratic", {})
        rep_totals = totals_for_party.get("Republican", {})

        dem_votes = 0
        rep_votes = 0
        dem_seats = 0
        rep_seats = 0
        total_population = 0

        for district_id in dem_totals:
            #total votes (dem and rep) from the same disctrict (inside the district plans

            total_population += dem_totals[district_id] + rep_totals.get(district_id, 0)


            dem_votes+= dem_totals[district_id]
            rep_votes+= rep_totals.get(district_id, 0)

            if dem_votes > rep_votes:
                dem_seats += 1
            elif rep_votes > dem_votes:
                rep_seats += 1


        total_plans+=1
        planId.append(total_plans)
        dem_share = dem_votes / total_population
        rep_share = rep_votes / total_population

        democratsPercentage.append(dem_share)
        republicanPercentage.append(rep_share)
        json["totalDistricts"] = total_district
        json["democratAvgVoteShare"] = (
            json["democratAvgVoteShare"] * (total_plans - 1) + dem_share
        ) / total_plans
        json["republicanAvgVoteShare"] = (
            json["republicanAvgVoteShare"] * (total_plans - 1) + rep_share
        ) / total_plans
        json["democratAvgSeatShare"] = (
            json["democratAvgSeatShare"] * (total_plans - 1) + dem_seats / len(dem_totals)
        ) / total_plans
        json["republicanAvgSeatShare"] = (
            json["republicanAvgSeatShare"] * (total_plans - 1) + rep_seats / len(rep_totals)
        ) / total_plans

 

        
        
      
    ensembles = [ {"planId": v1, "democratsPercentage":v2, "republicanPercentage": v3  }
                for v1 ,v2, v3 in zip(planId, democratsPercentage, republicanPercentage)]
    json["barData"].extend(ensembles) 
def calculate_opportunity(data, json, minority='hispanic',opportunity_threshold=0):

    results = []

    for district_plan in data['barData']:
        plan_id = district_plan['planId']
        opportunity_districts = 0

        for district in district_plan['districts']:
            total_population = district['white'] + district['black'] + district['asian'] + district['hispanic'] + district['other']
            minoirty_population = district[minority]

            minority_proportion = minoirty_population / total_population 
            
            
            
            if minority_proportion >= opportunity_threshold:
                opportunity_districts += 1
            

        for plan in json['barData']:
            if plan['planId'] == plan_id:
                plan['opportunityRepresentatives'] = opportunity_districts  # SMD: 1 rep per district
                plan['opportunityDistricts'] = opportunity_districts
                
                break
    return results
def collect_ensemble_demographic(data,shapefile ,json):
    if json["fips"] == "CO":
        hsp = "LAT"
    else:
        hsp = "HPAC"
    i = len(json.get("barData", []))
    planId = []
    white = []
    black = []
    asian = []
    hispanic = []
    other = []
    for item in data:
    
        districts =  item.get("ids", {})
        demo_summary= {
                    "white": 0,
                    "black": 0,
                    "asian": 0,
                    "hispanic": 0,
                    "other": 0
            }
        for district_id, precinct_ids  in districts.items():
       
            for precinct_id in precinct_ids:
                row = shapefile.iloc[precinct_id]
                demo_summary["white"] += row["WHT"]
                demo_summary["black"] += row["AFAM"]
                demo_summary["asian"] += row["ASN"]
                demo_summary["hispanic"] += row[hsp]
                demo_summary["other"] += row["OTHR"]
        i+=1
        planId.append(i)
        white.append(demo_summary["white"])
        black.append(demo_summary["black"])
        asian.append(demo_summary["asian"])
        hispanic.append(demo_summary["hispanic"])
        other.append(demo_summary["other"])

    ensembles = [ {"planId": v1, "white":v2, "black": v3, "asian": v4, "hispanic": v5, "other": v6  }
                for v1 ,v2, v3, v4, v5, v6 in zip(planId, white, black, asian, hispanic, other)]
    
    json["barData"].extend(ensembles)                 
def collect_district_plan_demographic(data, shapefile, json):

    i = len(json.get("barData", []))  # Start planId based on existing data

    for item in data:
        districts = item.get("ids", {})
        district_demographics = []

        for district_id, precinct_ids in districts.items():
            
            demo_summary = {
                "districtId": district_id,
                "white": 0,
                "black": 0,
                "asian": 0,
                "hispanic": 0,
                "other": 0
            }

            
            for precinct_id in precinct_ids:
                row = shapefile.iloc[precinct_id]
                demo_summary["white"] += row["WHT"]
                demo_summary["black"] += row["AFAM"]
                demo_summary["asian"] += row["ASN"]
                demo_summary["hispanic"] += row['LAT']
                demo_summary["other"] += (
                    (row["OTHR"] + row["AIAN"] + row["HPAC"]) if json["fips"] == "NV" 
                    else (row["OTHR"] ))
                

  
              

          
            district_demographics.append(demo_summary)

        
        i += 1
        json["barData"].append({
            "planId": i,
            "districts": district_demographics
        })
def collect_vote_seat_share_metrics(data, json):
    planId = []
    demVoteShare = []
    demSeatShare = []
    repSeatShare = []
    repVoteShare = []
    total_plans = len(json.get("barData", []))

    for item in data:
        total_districts = len(item.get("elec_res", {}).get("percents_for_party", {}).get("Democratic", {}))
        elec_res = item.get("elec_res", {})

        totals_for_party = elec_res.get("percents_for_party", {})
        dem_totals = totals_for_party.get("Democratic", {})
        rep_totals = totals_for_party.get("Republican", {})

        dem_seats = 0
        rep_seats = 0

        dem_vote_shares = []
        rep_vote_shares = []

        for district_id in dem_totals:
            dem_vote_share = dem_totals[district_id]
            rep_vote_share = rep_totals.get(district_id, 0)

            dem_vote_shares.append(dem_vote_share)
            rep_vote_shares.append(rep_vote_share)

            if dem_vote_share > rep_vote_share:
                dem_seats += 1
            elif rep_vote_share > dem_vote_share:
                rep_seats += 1

        avg_dem_vote_share = round(sum(dem_vote_shares) / total_districts, 2)
        avg_rep_vote_share = round(sum(rep_vote_shares) / total_districts, 2)
        dem_seat_share = round(dem_seats / total_districts, 2)
        rep_seat_share = round(rep_seats / total_districts, 2)

        
        total_plans += 1
        planId.append(total_plans)
        demVoteShare.append(avg_dem_vote_share)
        repVoteShare.append(avg_rep_vote_share)
        demSeatShare.append(dem_seat_share)
        repSeatShare.append(rep_seat_share)


    ensembles = [ 
                {"planId": v1,
                "demVoteShare":v2,
                "demSeatShare": v3 ,
                "repVoteShare": v4, 
                "repSeatShare": v5  }
                for v1 ,v2, v3 , v4,v5 in zip(planId, demVoteShare , demSeatShare, repVoteShare, repSeatShare)]

    json["barData"].extend(ensembles) 
def calculate_annotation_for_vote_seat_share(data): 
    bias = 0
    symmetry = 0
    responsiveness = 0

    dem_vote_share = np.array([entry['demVoteShare'] for entry in data['curveData']])
    rep_vote_share = np.array([entry['repVoteShare'] for entry in data['curveData']])
    total_districts = data['totalDistricts']
    
    # Calculate Democratic seat share cumulatively
    dem_seat_share = np.array([1 if vote > 0.5 else 0 for vote in dem_vote_share]).cumsum() / total_districts
    
    # Partisan Bias: Difference between seat share at 50% vote and 50%
    closest_to_50_index = np.argmin(np.abs(dem_vote_share - 0.5))
    bias = dem_seat_share[closest_to_50_index] - 0.5

    # Responsiveness: Average slope of the seats-votes curve in 45%-55% range
    in_range_indices = np.where((dem_vote_share >= 0.45) & (dem_vote_share <= 0.55))[0]
    slopes = []
    for i in range(len(in_range_indices) - 1):
        delta_vote = dem_vote_share[in_range_indices[i + 1]] - dem_vote_share[in_range_indices[i]]
        delta_seat = dem_seat_share[in_range_indices[i + 1]] - dem_seat_share[in_range_indices[i]]
        if delta_vote > 0:  # Avoid division by zero
            slopes.append(delta_seat / delta_vote)
    responsiveness = np.mean(slopes) if slopes else 0

    # Partisan Symmetry: Difference in seat shares at symmetric vote shares
    symmetry_differences = []
    for i, dem_vote in enumerate(dem_vote_share):
        symmetric_vote = 1 - dem_vote
        closest_symmetric_index = np.argmin(np.abs(dem_vote_share - symmetric_vote))
        if abs(dem_vote_share[closest_symmetric_index] - symmetric_vote) < 0.05:  # 5% tolerance
            sym_diff = dem_seat_share[i] - dem_seat_share[closest_symmetric_index]
            symmetry_differences.append(sym_diff)
    symmetry = np.mean(symmetry_differences) if symmetry_differences else 0

    
    data["bias"] = round(bias, 4)
    data["responsiveness"] = round(responsiveness, 4)
    data["symmetry"] = round(symmetry, 4)

    return data   
def collect_box_and_whisker_bins(data, json, races):

    total_bins = data["totalDistricts"]  # Total bins = total districts per plan

    
    bins_dict = {f"Bin {i+1}": {race: [] for race in races} for i in range(total_bins)}
    enacted_bins = {f"Bin {i+1}": {race: [] for race in races} for i in range(total_bins)}

    # Step 1: Process each district plan
    for plan_index, plan in enumerate(data["barData"]):  
        # Step 2: Extract and sort race percentages within each district plan
        race_percentages = {race: [] for race in races}
        for district in plan["districts"]:  
            total_population = sum(district[race] for race in races)
            for race in races:
                percentage = (district[race] / total_population) * 100 if total_population > 0 else 0
                race_percentages[race].append(percentage)
        
        # Sort percentages for each race and distribute them into bins
        for race in races:
            sorted_percentages = sorted(race_percentages[race])
            for i, value in enumerate(sorted_percentages):
                bin_key = f"Bin {i + 1}"
                bins_dict[bin_key][race].append(value)
        
        # Step 3: Process enacted plan separately (first plan only)
        if plan_index == 0:  
            for race in races:
                sorted_enacted = sorted(race_percentages[race])
                for i, value in enumerate(sorted_enacted):
                    bin_key = f"Bin {i + 1}"
                    enacted_bins[bin_key][race].append(value)

    # Step 4: Calculate statistics for each bin
    json["boxes"] = []
    for bin_key, race_data in bins_dict.items():
        stats = {"binNo": bin_key}
        for race, values in race_data.items():
            enacted_values = enacted_bins[bin_key][race]  # Enacted values for this bin and race
            values = np.array(values)
            if len(values) > 0:
                stats[race] = {
                    "min": float(np.min(values)),
                    "q1": float(np.percentile(values, 25)),
                    "median": float(np.median(values)),
                    "q3": float(np.percentile(values, 75)),
                    "max": float(np.max(values)),
                    "enactedValue": round(float(np.mean(enacted_values)), 2) if len(enacted_values) > 0 else 0
                }
            else:
                stats[race] = {"min": 0, "q1": 0, "median": 0, "q3": 0, "max": 0, "enactedValue": 0}
        json["boxes"].append(stats)
        json['totalDistricts'] = data["totalDistricts"]
def collec_metrics_plan_specific(data, json, plan_type):
    districtId = []
    democratsPercentage = []
    republicanPercentage = []
    opportunityThreshold = []
    isOpportunityDistrict = []
    white = []
    black = []
    asian = []
    hispanic = []
    other = []
    dem_votes = 0
    rep_votes = 0
    total_votes = []
    
    for item in data["features"]:
        districts = item["properties"]
        demographics =  districts["demographics"]
        demographic = districts.get("demographics", {})
        total_population = sum(demographic.values())
        districtId.append(districts.get("DISTRICTNO", None))
        democratsPercentage.append(districts.get("demRatio", 0))
        #republicanPercentage.append(districts.get("repRatio", 0))
        republicanPercentage.append(abs(1 - districts.get("demRatio" , 0)))
        opportunityThreshold.append(districts.get("oppThreshold", 0))
        isOpportunityDistrict.append(districts.get("oppDistrict", False))
        white.append(demographics.get("white", 0))
        black.append(demographics.get("black", 0))
        asian.append(demographics.get("asian", 0))
        hispanic.append(demographics.get("latin", 0))
        other.append(demographics.get("other", 0))
        total_votes.append(total_population)
        dem_votes += (total_population) * districts.get("demRatio", 0)
        rep_votes += (total_population) * (1 - districts.get("demRatio", 0))
    
   
    district_metrics = [
        {
            "districtId": dId,
            "democratsPercentage": dPct,
            "republicanPercentage": rPct,
            "opportunityThreshold": oThresh,
            "isOpportunityDistrict": isOpp,
            "white": w,
            "black": b,
            "asian": a,
            "hispanic": h,
            "other": o,
            "totalVotes": votes
        }
        for dId, dPct, rPct, oThresh, isOpp, w, b, a, h , o, votes, in zip(
            districtId, democratsPercentage, republicanPercentage,
            opportunityThreshold, isOpportunityDistrict,
            white, black, asian, hispanic,other, total_votes
        )
    ]

    json["districts"].extend(district_metrics)
    json["demTotalVotes"] = round(dem_votes)
    json["repTotalVotes"] = round(rep_votes)
    #json["planType"] = plan_type
def vote_seat_share_data_collectiotion_specific_plan(data ,json, planType ):

    districts = data["districts"]
    total_districts = data["totalDistricts"]
    curve_data = []
    dem_seats = 0
    rep_seats = 0


        

    for district in districts:
        dem_vote_share = district["democratsPercentage"]
        rep_vote_share = district["republicanPercentage"]

        
        if dem_vote_share > rep_vote_share:
            dem_seats +=1
        elif rep_vote_share > dem_vote_share:
            rep_seats +=1
   
        #dem_seat_share = round(dem_seats/ total_districts , 2)
        #rep_seat_share = round(rep_seats/ total_districts , 2)
            #"demSeatShare": dem_seat_share, "repSeatShare": rep_seat_share
 
        curve_data.append({
            "districtId": district["districtId"],
            "demVoteShare": dem_vote_share,
            "repVoteShare": rep_vote_share,
            "totalVotes": district["totalVotes"]
        
            
            
        })

    json["curveData"] = curve_data
    json["totalDistricts"] = len(districts)
    json["repTotalVotes"]= data["demTotalVotes"]
    json["demTotalVotes"]= data["repTotalVotes"]
    json["democratsSeats"]= dem_seats
    json["republicanSeats"]= rep_seats
def collect_result_collection_mmd(data, json):
    planId = []
    democratsPercentage = []
    republicanPercentage = []
    total_plans = len(json.get("barData", []))
    
    for item in data:
        # Initialize variables for the current plan
        total_districts = len(item.get("ids", {}))  # Total MMDs in this plan
        fra_res = item.get("fra_res", [])  # Vote results per district
        district_groupings = item.get("ids", {})  # MMD groupings 

        dem_votes = 0
        rep_votes = 0
        dem_seats = 0
        rep_seats = 0
        total_votes = 0

        # Process each MMD (big district) by aggregating sub-district results
        for mmd_id, sub_districts in district_groupings.items():
            total_dem_votes = 0
            total_rep_votes = 0

            # Sum votes for all sub-districts in this MMD
            for sub_district in sub_districts:
                for district_results in fra_res:
                    for key, votes in district_results.items():
                        if int(key[1:]) == sub_district:  # Match sub-district number
                            if key.startswith("D"):
                                total_dem_votes += votes
                            elif key.startswith("R"):
                                total_rep_votes += votes

            # Calculate vote shares for the MMD
            
            total_votes += total_dem_votes + total_rep_votes
            

            dem_votes+= total_dem_votes
            rep_votes+= total_rep_votes 


            # Determine seat winners based on the aggregated votes
            if total_dem_votes > total_rep_votes:
                dem_seats += 1
            elif total_rep_votes > total_dem_votes:
                rep_seats += 1

        # Update plan data
        total_plans += 1
        planId.append(total_plans)
        dem_share = dem_votes / total_votes
        rep_share = rep_votes / total_votes

        democratsPercentage.append(dem_share)
        republicanPercentage.append(rep_share)

        # Update averages in the JSON object
        json["democratAvgVoteShare"] = (
            json["democratAvgVoteShare"] * (total_plans - 1) + dem_share
        ) / total_plans
        json["republicanAvgVoteShare"] = (
            json["republicanAvgVoteShare"] * (total_plans - 1) + rep_share
        ) / total_plans
        json["democratAvgSeatShare"] = (
            json["democratAvgSeatShare"] * (total_plans - 1) + dem_seats / total_districts
        ) / total_plans
        json["republicanAvgSeatShare"] = (
            json["republicanAvgSeatShare"] * (total_plans - 1) + rep_seats / total_districts
        ) / total_plans

    # Append the results into barData
    ensembles = [{"planId": v1, "democratsPercentage": v2, "republicanPercentage": v3}
                 for v1, v2, v3 in zip(planId, democratsPercentage, republicanPercentage)]
    json["barData"].extend(ensembles)
    json['totalDistricts'] = total_districts
def collect_district_plan_demographic_mmd(data, shapefile, json):

    i = len(json.get("barData", []))

    # Create a lookup for demographics based on DISTRICTNO
    district_lookup = {
        feature["properties"]["DISTRICTNO"]: feature["properties"]["demographics"]
        for feature in shapefile["features"]
    }

    # Iterate over the district plans
    for item in data:
        districts = item.get("ids", {})  # Example: {"1": [1, 2, 3, 4], "2": [5, 6], "3": [7, 8, 9]}
        district_demographics = []

        # Process each MMD in the current district plan
        for mmd_id, sub_districts in districts.items():
            demo_summary = {
                "districtId": mmd_id,
                "white": 0,
                "black": 0,
                "asian": 0,
                "hispanic": 0,
                "other": 0,
                "representatives": len(sub_districts),
            }
            
          
            for sub_district_id in sub_districts:
                demographics = district_lookup.get(sub_district_id)
                if demographics:
                    demo_summary["white"] += demographics.get("white", 0)
                    demo_summary["black"] += demographics.get("black", 0)
                    demo_summary["asian"] += demographics.get("asian", 0)
                    demo_summary["hispanic"] += demographics.get("latin", 0)
                    demo_summary["other"] += demographics.get("other", 0)

            
            district_demographics.append(demo_summary)

        
        i += 1
        json["barData"].append({
            "planId": i,
            "districts": district_demographics
        })
        json["totalDistricts"] = len(districts)
def calculate_opportunity_mmd(data, json, minority='hispanic', opportunity_threshold=0.5):

    results = []

    for district_plan in data['barData']:
        plan_id = district_plan['planId']
        total_opportunity_reps = 0  # Track total opportunity representatives
        opportunity_districts = 0   # Track the number of districts meeting threshold

        # Iterate through all districts in the current plan
        for district in district_plan['districts']:
            # Fetch demographic values
            total_population = district['white'] + district['black'] + district['asian'] + district['hispanic']+ district['other']
            minority_population = district[minority]
            minority_proportion = minority_population / total_population if total_population > 0 else 0

            # Number of representatives in this district
            num_representatives = district.get("representatives", 1)  # Default to 1 if not provided
            

            if data.get("fips", "") == "CO":
                if num_representatives == 5:
                    threshold = 0.2
                elif num_representatives == 3:
                    threshold = 0.33
                else:
                    threshold = opportunity_threshold
            else:
                threshold = opportunity_threshold  
            #print(f"rep:{num_representatives} treh:{threshold} minnority:{minority_proportion}")     
            # Calculate opportunity representatives
            if minority_proportion >= threshold:
                opportunity_reps = num_representatives
                total_opportunity_reps += opportunity_reps
                opportunity_districts += 1  # Count this district as an opportunity district
            else:
                opportunity_reps = 0

        
        for plan in json['barData']:
            if plan['planId'] == plan_id:
                plan['opportunityRepresentatives'] = total_opportunity_reps
                plan['opportunityDistricts'] = opportunity_districts
                break

        results.append({
            "planId": plan_id,
            "opportunityRepresentatives": total_opportunity_reps,
            "opportunityDistricts": opportunity_districts
        })

    return results
def collect_box_and_whisker_bins_mmd(data, json, races):
    total_bins = data["totalDistricts"]  # Total bins = total districts per plan

    # Initialize bins for all plans and enacted plan
    bins_dict = {
        f"Bin {i+1}": {"representatives": 0, **{race: [] for race in races}} for i in range(total_bins)
    }
    enacted_bins = {
        f"Bin {i+1}": {"representatives": 0, **{race: [] for race in races}} for i in range(total_bins)
    }

    # Step 1: Process each district plan
    for plan_index, plan in enumerate(data["barData"]):
        race_percentages = {race: [] for race in races}
        representatives = []  # Track the representatives for each district

        # Step 2: Extract race percentages and representatives count
        for district in plan["districts"]:
            total_population = sum(district[race] for race in races)
            for race in races:
                percentage = (district[race] / total_population) * 100 if total_population > 0 else 0
                race_percentages[race].append(percentage)

            # Track representatives for each district
            num_representatives = district.get("representatives", 1)
            representatives.append(num_representatives)

        # Step 3: Sort race percentages and distribute into bins
        for race in races:
            sorted_percentages = sorted(race_percentages[race])
            for i, value in enumerate(sorted_percentages):
                bin_key = f"Bin {i + 1}"
                bins_dict[bin_key][race].append(value)
                bins_dict[bin_key]["representatives"] = representatives[i]  # Set representatives

        # Step 4: Process enacted plan 
        if plan_index == 0:
            for race in races:
                sorted_enacted = sorted(race_percentages[race])
                for i, value in enumerate(sorted_enacted):
                    bin_key = f"Bin {i + 1}"
                    enacted_bins[bin_key][race].append(value)
                    enacted_bins[bin_key]["representatives"] = representatives[i]  # Set representatives

    # Step 5: Calculate statistics for each bin
    json["boxes"] = []
    for bin_key, race_data in bins_dict.items():
        stats = {"binNo": bin_key, "totalRepresentatives": race_data["representatives"]}
        for race, values in race_data.items():
            if race == "representatives":
                continue  # Skip processing representatives here
            
            enacted_values = enacted_bins[bin_key][race]  # Enacted values for this bin and race
            values = np.array(values)
            if len(values) > 0:
                stats[race] = {
                    "min": float(np.min(values)),
                    "q1": float(np.percentile(values, 25)),
                    "median": float(np.median(values)),
                    "q3": float(np.percentile(values, 75)),
                    "max": float(np.max(values)),
                    "enactedValue": round(float(np.mean(enacted_values)), 2) if len(enacted_values) > 0 else 0
                }
            else:
                stats[race] = {"min": 0, "q1": 0, "median": 0, "q3": 0, "max": 0, "enactedValue": 0}
        
        json["boxes"].append(stats)
    
    json['totalDistricts'] = data["totalDistricts"]
def plan_specific_summary_data(data , json):
    
    crs = data["crs"]
    properties = crs["properties"]
    safeDistricts = properties.get("safeDistricts") 
    totalDistricts = properties.get("noOfDistricts")
    totalOpportunityDistricts = properties.get("oppDistricts") 
    district_plan_total_population = 0
    total_representatives = 0
    representatives = []
    isOpportunityDistrict = []
    opportunityThreshold = []
    districtId = []
    democratsPercentage = []
    republicanPercentage = []
    district_total_population = []
    district_winner = []
    white = []
    black = []
    asian = []
    hispanic = []
    other = []
    winner = None
    total_dem_votes = 0
    total_rep_votes = 0
    for item in data["features"]:
        districts = item["properties"]
        total_representatives += districts.get("representatives")
        representatives.append(districts.get("representatives", 0))

        
        demographics = districts.get("demographics", {})
        total_population = sum(demographics.values())
        district_plan_total_population += total_population
        district_total_population.append(total_population)
        democratsPercentage.append(districts.get("demRatio", 0))
        republicanPercentage.append(abs(1 - districts.get("demRatio" , 0)))
        opportunityThreshold.append(districts.get("oppThreshold", 0))
        isOpportunityDistrict.append(districts.get("oppDistrict", False))
        dem_ratio = districts.get("demRatio", 0)

        dem_votes = total_population * dem_ratio
        rep_votes = total_population * (1 - dem_ratio)

        districtId.append(districts.get("DISTRICTNO", None))
        white.append(demographics.get("white", 0))
        black.append(demographics.get("black", 0))
        asian.append(demographics.get("asian", 0))
        hispanic.append(demographics.get("latin", 0))
        other.append(demographics.get("other", 0))
        district_winner.append("DEMOCRAT" if dem_ratio > 0.5 else "REPUBLICAN")

        total_dem_votes += dem_votes
        total_rep_votes += rep_votes
                               

    district_metrics = [
        {
            "districtId": dId,
            "democratsPercentage": dPct,
            "republicanPercentage": rPct,
            "totalRepresentatives": rep,
            "totalPopulation" : pop,
            "opportunityThreshold": oThresh,
            "isOpportunityDistrict": isOpp,
            "white": w,
            "black": b,
            "asian": a,
            "hispanic": h,
            "other": o,
            "partyWinner": win
            
        }
        for dId, dPct, rPct, rep, pop, oThresh, isOpp, w, b, a, h ,o,win in zip(
            districtId, democratsPercentage, republicanPercentage, representatives,
            district_total_population, opportunityThreshold, isOpportunityDistrict,
            white, black, asian, hispanic, other, district_winner
        )
    ]
    winner = "DEMOCRAT" if total_dem_votes > total_rep_votes else "REPUBLICAN"
    
    json["totalDistricts"] = totalDistricts 
    json["totalPopulation"] = district_plan_total_population
    json["partyWinner"] = winner
    json["safeDistricts"] = safeDistricts
    json["totalRepresentative"] = total_representatives
    json["totalOpportunityDistricts"] = totalOpportunityDistricts
    json["districts_summary"].extend(district_metrics)


# NEVADA

In [288]:
#SMD
shp_path = 'Nevada/raw_data/nv_2020_final.shp'
shapefile = gpd.read_file(shp_path)
shapefile = shapefile.to_crs(epsg=4326)
shapefile['ASN'] = shapefile['ASN'].fillna(0)
shapefile['AFAM'] = shapefile['AFAM'].fillna(0)
shapefile['WHT'] = shapefile['WHT'].fillna(0)
shapefile['HPAC'] = shapefile['HPAC'].fillna(0)
shapefile['OTHR'] = shapefile['OTHR'].fillna(0)
shapefile['LAT'] = shapefile['LAT'].fillna(0)
shapefile['AIAN'] = shapefile['AIAN'].fillna(0)
#smd data load
smd_data = ["Nevada/raw_data/smd_random_district0.json", "Nevada/raw_data/NV_smd_random_district1.json", "Nevada/raw_data/NV_smd_random_district2.json","Nevada/raw_data/NV_smd_random_district3.json"]
races = ["white", "black", "asian", "hispanic", "other"]

nv_demographic_data = {
    "fips": "NV",
    "electionType": "SMD",
    "totalDistricts": 4,
    "barData": []
    }
nv_data = {
    "fips": "NV",
    "electionType": "SMD",
    "totalDistricts": 4,
    "democratAvgVoteShare": 0,
    "democratAvgSeatShare": 0,
    "republicanAvgVoteShare": 0,
    "republicanAvgSeatShare": 0,
    "barData": []

    }

'''


#load ensamble data
for ensemble_file_path in smd_data:
    with open(ensemble_file_path, 'r') as file:
        smd_ensemble = json.load(file)
        collect_district_plan_demographic(smd_ensemble,shapefile ,nv_demographic_data)
        election_result_collection(smd_ensemble, nv_data)


calculate_opportunity(nv_demographic_data, nv_data, 'hispanic', 0.50)



with open("Nevada/smd_ensemble/nv_smd_ensemble_bar.json", 'w') as json_file:
    json.dump(nv_data, json_file, indent=4)

'''
nv_box_and_whisker_data = {
    "fips": "NV",
    "electionType": "SMD",
    "totalDistricts": 4,
    "boxes": []

    }

collect_box_and_whisker_bins(nv_demographic_data, nv_box_and_whisker_data, races)   
with open("Nevada/smd_ensemble/nv_smd_ensemble_box_and_whisker.json", 'w') as json_file:
    json.dump(nv_box_and_whisker_data, json_file, indent=4)


smd_plan_specific = ["Nevada/raw_data/nv_smd_average0.json",
                     "Nevada/raw_data/nv_smd_demfavored0.json",
                     "Nevada/raw_data/nv_smd_fair0.json",
                     "Nevada/raw_data/nv_smd_repfavored0.json",
                     "Nevada/raw_data/NV_SMD_ENACTED.json"]

smd_interesting_plans = ["AVERAGE","DEMFAVORED", "FAIR","REPFAVORED", "ENACTED"]

smd_interesting_plan_data = []

#load specific plan data data
for plan_specific_file_path in smd_plan_specific:
    with open(plan_specific_file_path, 'r') as file:
        smd_plan_specific = json.load(file)
        smd_interesting_plan_data.append(smd_plan_specific)


for plan_name, plan_data in zip(smd_interesting_plans,smd_interesting_plan_data):
    nv_plan_specific_data = {
        "fips": "NV",
        "electionType": "SMD",
        "totalDistricts": 4,
        "characteristic": plan_name,
        "demTotalVotes": 0,
        "repTotalVotes": 0,
        "districts": []
        }
    collec_metrics_plan_specific(plan_data, nv_plan_specific_data, plan_name)
   


    nv_plan_specific = {
        "fips": "NV",
        "electionType": "SMD",
        "totalDistricts": 4,
        "characteristic": plan_name,
        "democratsSeats": 0,
        "republicanSeats": 0,
        "demTotalVotes": 0,
        "repTotalVotes": 0,
        "bias": 0,
        "symmetry": 0,
        "responsiveness": 0,
        "curveData": []
        }
    vote_seat_share_data_collectiotion_specific_plan( nv_plan_specific_data ,nv_plan_specific, plan_name)
    calculate_annotation_for_vote_seat_share(nv_plan_specific)
    
    file_path = f'Nevada/smd_plan_specific/nv_smd_{plan_name}.json'
    with open(file_path, 'w') as json_file:
        json.dump(nv_plan_specific, json_file, indent=4)

  

In [287]:

shp_path = 'Nevada/raw_data/nv_2020_final.shp'
shapefile = gpd.read_file(shp_path)
shapefile = shapefile.to_crs(epsg=4326)
shapefile['ASN'] = shapefile['ASN'].fillna(0)
shapefile['AFAM'] = shapefile['AFAM'].fillna(0)
shapefile['WHT'] = shapefile['WHT'].fillna(0)
shapefile['HPAC'] = shapefile['HPAC'].fillna(0)
shapefile['OTHR'] = shapefile['OTHR'].fillna(0)
shapefile['LAT'] = shapefile['LAT'].fillna(0)
shapefile['AIAN'] = shapefile['AIAN'].fillna(0)
shapefile['other'] = shapefile['AIAN'] + shapefile['OTHR']

test_df = shapefile[['WHT', 'AFAM', 'ASN', 'HPAC','LAT','other']].sum().reset_index()
test_df.columns = ['group', 'population']
total_population = test_df['population'].sum()
test_df['percentage'] = test_df.apply(lambda x: x['population'] / total_population * 100, axis = 1)
test_df

,group,population,percentage
0,WHT,1418781.0,43.929406
1,AFAM,320206.0,9.914468
2,ASN,318407.0,9.858766
3,HPAC,45733.0,1.416021
4,LAT,703372.0,21.778354
5,other,423185.0,13.102985


In [340]:
#MMD
shp_path = 'Nevada/raw_data/nv_2020_final.shp'
with open("Nevada/raw_data/NV_SMD_ENACTED.json", 'r') as file:
    enacted_plan = json.load(file)
shapefile = gpd.read_file(shp_path)
shapefile = shapefile.to_crs(epsg=4326)
shapefile['ASN'] = shapefile['ASN'].fillna(0)
shapefile['AFAM'] = shapefile['AFAM'].fillna(0)
shapefile['WHT'] = shapefile['WHT'].fillna(0)
shapefile['HPAC'] = shapefile['HPAC'].fillna(0)
shapefile['OTHR'] = shapefile['OTHR'].fillna(0)
shapefile['LAT'] = shapefile['LAT'].fillna(0)
shapefile['AIAN'] = shapefile['AIAN'].fillna(0)

races = ["white", "black", "asian", "hispanic", "other"]
mmd_data = ["Nevada/raw_data/mmd_random_district0.json", "Nevada/raw_data/NV_mmd_random_district1.json", "Nevada/raw_data/NV_mmd_random_district2.json", "Nevada/raw_data/NV_mmd_random_district3.json"]

nv_mmd_demographic_data = {
    "fips": "NV",
    "electionType": "MMD",
    "totalDistricts": 0,
    "barData": []
    }
nv_mmd_data = {
    "fips": "NV",
    "electionType": "MMD",
    "totalDistricts": 0,
    "democratAvgVoteShare": 0,
    "democratAvgSeatShare": 0,
    "republicanAvgVoteShare": 0,
    "republicanAvgSeatShare": 0,
    "barData": []
}

#load mmd ensemble data    
for ensemble_file_path in mmd_data:
    with open(ensemble_file_path, 'r') as file:
        mmd_ensemble = json.load(file)
        collect_district_plan_demographic_mmd(mmd_ensemble,enacted_plan ,nv_mmd_demographic_data)
        election_result_collection(mmd_ensemble, nv_mmd_data)

calculate_opportunity_mmd(nv_mmd_demographic_data, nv_mmd_data, 'hispanic', 0.25)


with open("Nevada/mmd_ensemble/nv_mmd_ensemble_bar.json", 'w') as json_file:
    json.dump(nv_mmd_data, json_file, indent=4)
'''
nv_mmd_box_and_whisker_data = {
    "fips": "NV",
    "electionType": "MMD",
    "totalDistricts": 0,
    "boxes": []

    }

collect_box_and_whisker_bins(nv_mmd_demographic_data, nv_mmd_box_and_whisker_data, races)   
with open("Nevada/mmd_ensemble/nv_mmd_ensemble_box_and_whisker.json", 'w') as json_file:
    json.dump(nv_mmd_box_and_whisker_data, json_file, indent=4)


mmd_plan_specific = ["Nevada/raw_data/nv_mmd_average0.json",
                     "Nevada/raw_data/nv_mmd_demfavored0.json" ,
                     "Nevada/raw_data/nv_mmd_fair0.json", 
                     "Nevada/raw_data/nv_mmd_repfavored0.json"]
mmd_interesting_plans = ["AVERAGE","DEMFAVORED", "FAIR","REPFAVORED"]

mmd_interesting_plan_data = []

#load specific plan mmd data data
for plan_specific_file_path in mmd_plan_specific:
    with open(plan_specific_file_path, 'r') as file:
        mmd_plan_specific = json.load(file)
        mmd_interesting_plan_data.append(mmd_plan_specific)


for plan_name, plan_data in zip(mmd_interesting_plans,mmd_interesting_plan_data):
    nv_plan_specific_data = {
        "fips": "NV",
        "electionType": "MMD",
        "totalDistricts": 4,
        "characteristic": plan_name,
        "demTotalVotes": 0,
        "repTotalVotes": 0,
        "districts": []
        }
    collec_metrics_plan_specific(plan_data, nv_plan_specific_data, plan_name)

    nv_plan_specific = {
        "fips": "NV",
        "electionType": "MMD",
        "totalDistricts": 4,
        "characteristic": plan_name,
        "democratsSeats": 0,
        "republicanSeats": 0,
        "demTotalVotes": 0,
        "repTotalVotes": 0,
        "bias": 0,
        "symmetry": 0,
        "responsiveness": 0,
        "curveData": []
        }
    vote_seat_share_data_collectiotion_specific_plan( nv_plan_specific_data ,nv_plan_specific, plan_name)
    calculate_annotation_for_vote_seat_share(nv_plan_specific)
    
    file_path = f'Nevada/mmd_plan_specific/nv_mmd_{plan_name}.json'
    with open(file_path, 'w') as json_file:
        json.dump(nv_plan_specific, json_file, indent=4)
'''


'\nnv_mmd_box_and_whisker_data = {\n    "fips": "NV",\n    "electionType": "MMD",\n    "totalDistricts": 0,\n    "boxes": []\n\n    }\n\ncollect_box_and_whisker_bins(nv_mmd_demographic_data, nv_mmd_box_and_whisker_data, races)   \nwith open("Nevada/mmd_ensemble/nv_mmd_ensemble_box_and_whisker.json", \'w\') as json_file:\n    json.dump(nv_mmd_box_and_whisker_data, json_file, indent=4)\n\n\nmmd_plan_specific = ["Nevada/raw_data/nv_mmd_average0.json",\n                     "Nevada/raw_data/nv_mmd_demfavored0.json" ,\n                     "Nevada/raw_data/nv_mmd_fair0.json", \n                     "Nevada/raw_data/nv_mmd_repfavored0.json"]\nmmd_interesting_plans = ["AVERAGE","DEMFAVORED", "FAIR","REPFAVORED"]\n\nmmd_interesting_plan_data = []\n\n#load specific plan mmd data data\nfor plan_specific_file_path in mmd_plan_specific:\n    with open(plan_specific_file_path, \'r\') as file:\n        mmd_plan_specific = json.load(file)\n        mmd_interesting_plan_data.append(mmd_plan_specific)\n

In [272]:
shp_path = 'Colorado/raw_data/district_data.shp'
shapefile = gpd.read_file(shp_path)
shapefile = shapefile.to_crs(epsg=4326)
shapefile['other'] =  shapefile['OTHR']

test_df = shapefile[['WHT', 'AFAM', 'ASN', 'LAT','other']].sum().reset_index()
test_df.columns = ['group', 'population']
total_population = test_df['population'].sum()
test_df['percentage'] = test_df.apply(lambda x: x['population'] / total_population * 100, axis = 1)
test_df

,group,population,percentage
0,WHT,3519718.0,70.125806
1,AFAM,187399.0,3.733681
2,ASN,134105.0,2.671868
3,LAT,1037234.0,20.665539
4,other,140692.0,2.803105


# COLORADO 

In [333]:
#SMD
shp_path = 'Colorado/raw_data/district_data.shp'
shapefile = gpd.read_file(shp_path)
shapefile = shapefile.to_crs(epsg=4326)
shapefile['ASN'] = shapefile['ASN'].fillna(0)
shapefile['AFAM'] = shapefile['AFAM'].fillna(0)
shapefile['WHT'] = shapefile['WHT'].fillna(0)
shapefile['LAT'] = shapefile['LAT'].fillna(0)
shapefile['OTHR'] = shapefile['OTHR'].fillna(0)

#smd data load
smd_data = ["Colorado/raw_data/CO_smd_random_district0.json", "Colorado/raw_data/CO_smd_random_district1.json","Colorado/raw_data/CO_smd_random_district2.json", "Colorado/raw_data/CO_smd_random_district3.json"]
races = ["white", "black", "asian", "hispanic", 'other']

co_demographic_data = {
    "fips": "CO",
    "electionType": "SMD",
    "totalDistricts": 8,
    "barData": []
    }
co_data = {
    "fips": "CO",
    "electionType": "SMD",
    "totalDistricts": 0,
    "democratAvgVoteShare": 0,
    "democratAvgSeatShare": 0,
    "republicanAvgVoteShare": 0,
    "republicanAvgSeatShare": 0,
    "barData": []

    }


#load ensamble data
for ensemble_file_path in smd_data:
    with open(ensemble_file_path, 'r') as file:
        smd_ensemble = json.load(file)
        collect_district_plan_demographic(smd_ensemble,shapefile ,co_demographic_data)
        election_result_collection(smd_ensemble, co_data)

calculate_opportunity(co_demographic_data, co_data, 'hispanic', 0.50)



with open("Colorado/smd_ensemble/co_smd_ensemble_bar.json", 'w') as json_file:
    json.dump(co_data, json_file, indent=4)


co_box_and_whisker_data = {
    "fips": "CO",
    "electionType": "SMD",
    "totalDistricts": 8,
    "boxes": []

    }

collect_box_and_whisker_bins(co_demographic_data, co_box_and_whisker_data, races)   
with open("Colorado/smd_ensemble/co_smd_ensemble_box_and_whisker.json", 'w') as json_file:
    json.dump(co_box_and_whisker_data, json_file, indent=4)

'''
smd_plan_specific = ["Colorado/raw_data/CO_SMD_average0.json",
                     "Colorado/raw_data/CO_SMD_demfavored0.json",
                     "Colorado/raw_data/CO_SMD_fair0.json",
                     "Colorado/raw_data/CO_SMD_repfavored0.json",
                     "Colorado/raw_data/CO_SMD_ENACTED.json"]

smd_interesting_plans = ["AVERAGE","DEMFAVORED", "FAIR","REPFAVORED", "ENACTED"]

smd_interesting_plan_data = []

#load specific plan data data
for plan_specific_file_path in smd_plan_specific:
    with open(plan_specific_file_path, 'r') as file:
        smd_plan_specific = json.load(file)
        smd_interesting_plan_data.append(smd_plan_specific)


for plan_name, plan_data in zip(smd_interesting_plans,smd_interesting_plan_data):
    co_plan_specific_data = {
        "fips": "CO",
        "electionType": "SMD",
        "totalDistricts": 8,
        "characteristic": plan_name,
        "demTotalVotes": 0,
        "repTotalVotes": 0,
        "districts": []
        }
    collec_metrics_plan_specific(plan_data, co_plan_specific_data, plan_name)
   


    co_plan_specific = {
        "fips": "CO",
        "electionType": "SMD",
        "totalDistricts": 8,
        "characteristic": plan_name,
        "democratsSeats": 0,
        "republicanSeats": 0,
        "demTotalVotes": 0,
        "repTotalVotes": 0,
        "bias": 0,
        "symmetry": 0,
        "responsiveness": 0,
        "curveData": []
        }
    vote_seat_share_data_collectiotion_specific_plan( co_plan_specific_data ,co_plan_specific, plan_name)
    calculate_annotation_for_vote_seat_share(co_plan_specific)
    
    file_path = f'Colorado/smd_plan_specific/co_smd_{plan_name}.json'
    with open(file_path, 'w') as json_file:
        json.dump(co_plan_specific, json_file, indent=4)
'''



'\nsmd_plan_specific = ["Colorado/raw_data/CO_SMD_average0.json",\n                     "Colorado/raw_data/CO_SMD_demfavored0.json",\n                     "Colorado/raw_data/CO_SMD_fair0.json",\n                     "Colorado/raw_data/CO_SMD_repfavored0.json",\n                     "Colorado/raw_data/CO_SMD_ENACTED.json"]\n\nsmd_interesting_plans = ["AVERAGE","DEMFAVORED", "FAIR","REPFAVORED", "ENACTED"]\n\nsmd_interesting_plan_data = []\n\n#load specific plan data data\nfor plan_specific_file_path in smd_plan_specific:\n    with open(plan_specific_file_path, \'r\') as file:\n        smd_plan_specific = json.load(file)\n        smd_interesting_plan_data.append(smd_plan_specific)\n\n\nfor plan_name, plan_data in zip(smd_interesting_plans,smd_interesting_plan_data):\n    co_plan_specific_data = {\n        "fips": "CO",\n        "electionType": "SMD",\n        "totalDistricts": 8,\n        "characteristic": plan_name,\n        "demTotalVotes": 0,\n        "repTotalVotes": 0,\n        "dis

In [342]:
#MMD
shp_path = 'Colorado/raw_data/district_data.shp'
with open("Colorado/raw_data/CO_SMD_ENACTED.json", 'r') as file:
    enacted_plan = json.load(file)
shapefile = gpd.read_file(shp_path)
shapefile = shapefile.to_crs(epsg=4326)
shapefile['ASN'] = shapefile['ASN'].fillna(0)
shapefile['AFAM'] = shapefile['AFAM'].fillna(0)
shapefile['WHT'] = shapefile['WHT'].fillna(0)
shapefile['LAT'] = shapefile['LAT'].fillna(0)
shapefile['OTHR'] = shapefile['OTHR'].fillna(0)

races = ["white", "black", "asian", "hispanic", 'other']
mmd_data = ["Colorado/raw_data/CO_mmd_random_district0.json", "Colorado/raw_data/CO_mmd_random_district1.json", "Colorado/raw_data/CO_mmd_random_district2.json", "Colorado/raw_data/CO_mmd_random_district3.json"]

co_mmd_demographic_data = {
    "fips": "CO",
    "electionType": "MMD",
    "totalDistricts": 0,
    "barData": []
    }
co_mmd_data = {
    "fips": "CO",
    "electionType": "MMD",
    "totalDistricts": 0,
    "democratAvgVoteShare": 0,
    "democratAvgSeatShare": 0,
    "republicanAvgVoteShare": 0,
    "republicanAvgSeatShare": 0,
    "barData": []
}

#load mmd ensemble data    
for ensemble_file_path in mmd_data:
    with open(ensemble_file_path, 'r') as file:
        mmd_ensemble = json.load(file)
        collect_district_plan_demographic_mmd(mmd_ensemble,enacted_plan ,co_mmd_demographic_data)
        election_result_collection(mmd_ensemble, co_mmd_data)

calculate_opportunity_mmd(co_mmd_demographic_data, co_mmd_data, 'hispanic', 0.20)


with open("Colorado/mmd_ensemble/co_mmd_ensemble_bar.json", 'w') as json_file:
    json.dump(co_mmd_data, json_file, indent=4)

co_mmd_box_and_whisker_data = {
    "fips": "CO",
    "electionType": "MMD",
    "totalDistricts": 0,
    "boxes": []

    }

'''
collect_box_and_whisker_bins_mmd(co_mmd_demographic_data, co_mmd_box_and_whisker_data, races)   
with open("Colorado/mmd_ensemble/co_mmd_ensemble_box_and_and_whisker.json", 'w') as json_file:
    json.dump(co_mmd_box_and_whisker_data, json_file, indent=4)


mmd_plan_specific = ["Colorado/raw_data/CO_mmd_AVERAGE0.json",
                     "Colorado/raw_data/CO_mmd_DEMFAVORED0.json" ,
                     "Colorado/raw_data/CO_mmd_FAIR0.json", 
                     "Colorado/raw_data/CO_mmd_repfavored0.json"]
mmd_interesting_plans = ["AVERAGE","DEMFAVORED", "FAIR","REPFAVORED"]

mmd_interesting_plan_data = []

#load specific plan mmd data data
for plan_specific_file_path in mmd_plan_specific:
    with open(plan_specific_file_path, 'r') as file:
        mmd_plan_specific = json.load(file)
        mmd_interesting_plan_data.append(mmd_plan_specific)


for plan_name, plan_data in zip(mmd_interesting_plans,mmd_interesting_plan_data):
    co_plan_specific_data = {
        "fips": "CO",
        "electionType": "MMD",
        "totalDistricts": 0,
        "characteristic": plan_name,
        "demTotalVotes": 0,
        "repTotalVotes": 0,
        "districts": []
        }
    collec_metrics_plan_specific(plan_data, co_plan_specific_data, plan_name)

    co_plan_specific = {
        "fips": "CO",
        "electionType": "MMD",
        "totalDistricts": 0,
        "characteristic": plan_name,
        "democratsSeats": 0,
        "republicanSeats": 0,
        "demTotalVotes": 0,
        "repTotalVotes": 0,
        "bias": 0,
        "symmetry": 0,
        "responsiveness": 0,
        "curveData": []
        }
    vote_seat_share_data_collectiotion_specific_plan(co_plan_specific_data ,co_plan_specific, plan_name)
    calculate_annotation_for_vote_seat_share(co_plan_specific)
    
    file_path = f'Colorado/mmd_plan_specific/co_mmd_{plan_name}.json'
    with open(file_path, 'w') as json_file:
        json.dump(co_plan_specific, json_file, indent=4)
'''


'\ncollect_box_and_whisker_bins_mmd(co_mmd_demographic_data, co_mmd_box_and_whisker_data, races)   \nwith open("Colorado/mmd_ensemble/co_mmd_ensemble_box_and_and_whisker.json", \'w\') as json_file:\n    json.dump(co_mmd_box_and_whisker_data, json_file, indent=4)\n\n\nmmd_plan_specific = ["Colorado/raw_data/CO_mmd_AVERAGE0.json",\n                     "Colorado/raw_data/CO_mmd_DEMFAVORED0.json" ,\n                     "Colorado/raw_data/CO_mmd_FAIR0.json", \n                     "Colorado/raw_data/CO_mmd_repfavored0.json"]\nmmd_interesting_plans = ["AVERAGE","DEMFAVORED", "FAIR","REPFAVORED"]\n\nmmd_interesting_plan_data = []\n\n#load specific plan mmd data data\nfor plan_specific_file_path in mmd_plan_specific:\n    with open(plan_specific_file_path, \'r\') as file:\n        mmd_plan_specific = json.load(file)\n        mmd_interesting_plan_data.append(mmd_plan_specific)\n\n\nfor plan_name, plan_data in zip(mmd_interesting_plans,mmd_interesting_plan_data):\n    co_plan_specific_data = {

In [325]:
co_mmd_demographic_data


{'fips': 'CO',
 'electionType': 'MMD',
 'totalDistricts': 2,
 'barData': [{'planId': 1,
   'districts': [{'districtId': '1',
     'white': 1303706,
     'black': 48074,
     'asian': 50136,
     'hispanic': 382350,
     'other': 54971,
     'representatives': 3},
    {'districtId': '2',
     'white': 2216012,
     'black': 139325,
     'asian': 83969,
     'hispanic': 654884,
     'other': 85721,
     'representatives': 5}]},
  {'planId': 2,
   'districts': [{'districtId': '1',
     'white': 1307417,
     'black': 50829,
     'asian': 48722,
     'hispanic': 393948,
     'other': 56034,
     'representatives': 3},
    {'districtId': '2',
     'white': 2212301,
     'black': 136570,
     'asian': 85383,
     'hispanic': 643286,
     'other': 84658,
     'representatives': 5}]},
  {'planId': 3,
   'districts': [{'districtId': '1',
     'white': 1361947,
     'black': 102663,
     'asian': 61578,
     'hispanic': 310776,
     'other': 63749,
     'representatives': 3},
    {'districtId': 

# ADDITIONAL JSONS

In [344]:
'''
gui 6
he user should have the option to view the detailed election results of an “interesting” simulated election in an ensemble. 
Details include:
 the district number, 
 number of representatives for that district,
 party affiliation, etc.
results for all candidates, winners, and losers, along with vote totals.
gui 11
When the user clicks on a district plan shown in the menu of available plans, details about the selected district plan will be displayed. 
Details include :
number of districts 
and summary of each district in the plan (including population and population by demographic group), 
Republican/Democratic split, 
election from which voting preference is used, 
number of opportunity districts, 
threshold used for opportunity district calculation,
 and number of safe districts.
'''
#nevada 
#smd
'''
smd_plan_specific = ["Nevada/raw_data/nv_smd_average0.json",
                     "Nevada/raw_data/nv_smd_demfavored0.json",
                     "Nevada/raw_data/nv_smd_fair0.json",
                     "Nevada/raw_data/nv_smd_repfavored0.json",
                     "Nevada/raw_data/NV_SMD_ENACTED.json"]

smd_interesting_plans = ["AVERAGE","DEMFAVORED", "FAIR","REPFAVORED", "ENACTED"]

smd_interesting_plan_data = []


for plan_specific_file_path in smd_plan_specific:
    with open(plan_specific_file_path, 'r') as file:
        smd_plan_specific = json.load(file)
        smd_interesting_plan_data.append(smd_plan_specific)

smd_summary_data_plan_specific = {
     "fips": "NV",
     "electionType": "SMD",
     "interesting_plans_summary": []

} 
for plan_name , plan_data in zip(smd_interesting_plans, smd_interesting_plan_data):
        nv_plan_specific_data = {
        "characteristic": plan_name,
        "partyWinner": None,
        "totalDistricts": 0,
        "totalPopulation": 0,
        "partyWinner": 0,
        "safeDistricts": 0,
        "totalRepresentative": 0,
        "totalOpportunityDistricts": 0,
        "districts_summary": []

        }

        plan_specific_summary_data(plan_data, nv_plan_specific_data)
        smd_summary_data_plan_specific["interesting_plans_summary"].append(nv_plan_specific_data)

with open("Nevada/ensemble_summary/nv_smd_summary_data_plan_specific.json", 'w') as json_file:
    json.dump(smd_summary_data_plan_specific, json_file, indent=4)
'''
#mmd 
mmd_plan_specific = ["Nevada/raw_data/nv_mmd_average0.json",
                     "Nevada/raw_data/nv_mmd_demfavored0.json" ,
                     "Nevada/raw_data/nv_mmd_fair0.json", 
                     "Nevada/raw_data/nv_mmd_repfavored0.json"]
mmd_interesting_plans = ["AVERAGE","DEMFAVORED", "FAIR","REPFAVORED"]

mmd_interesting_plan_data = []
mmd_summary_data_plan_specific = {
     "fips": "NV",
     "electionType": "MMD",
     "interesting_plans_summary": []

} 

#load specific plan mmd data data
for plan_specific_file_path in mmd_plan_specific:
    with open(plan_specific_file_path, 'r') as file:
        mmd_plan_specific = json.load(file)
        mmd_interesting_plan_data.append(mmd_plan_specific)


for plan_name , plan_data in zip(mmd_interesting_plans, mmd_interesting_plan_data):
        nv_plan_specific_data = {
        "characteristic": plan_name,
        "partyWinner": None,
        "totalDistricts": 0,
        "totalPopulation": 0,
        "partyWinner": 0,
        "safeDistricts": 0,
        "totalRepresentative": 0,
        "totalOpportunityDistricts": 0,
        "districts_summary": []


        }

        plan_specific_summary_data(plan_data, nv_plan_specific_data)
        mmd_summary_data_plan_specific["interesting_plans_summary"].append(nv_plan_specific_data)

with open("Nevada/ensemble_summary/nv_mmd_summary_data_plan_specific.json", 'w') as json_file:
    json.dump(mmd_summary_data_plan_specific, json_file, indent=4)




In [217]:
#colorado 
#smd
smd_plan_specific = ["Colorado/raw_data/CO_SMD_average0.json",
                     "Colorado/raw_data/CO_SMD_demfavored0.json",
                     "Colorado/raw_data/CO_SMD_fair0.json",
                     "Colorado/raw_data/CO_SMD_repfavored0.json",
                     "Colorado/raw_data/CO_SMD_ENACTED.json"]

smd_interesting_plans = ["AVERAGE","DEMFAVORED", "FAIR","REPFAVORED", "ENACTED"]

smd_interesting_plan_data = []

#load specific plan data data
for plan_specific_file_path in smd_plan_specific:
    with open(plan_specific_file_path, 'r') as file:
        smd_plan_specific = json.load(file)
        smd_interesting_plan_data.append(smd_plan_specific)

smd_summary_data_plan_specific = {
     "fips": "CO",
     "electionType": "SMD",
     "interesting_plans_summary": []

} 
for plan_name , plan_data in zip(smd_interesting_plans, smd_interesting_plan_data):
        nv_plan_specific_data = {
        "characteristic": plan_name,
        "partyWinner": None,
        "totalDistricts": 0,
        "totalPopulation": 0,
        "partyWinner": 0,
        "safeDistricts": 0,
        "totalRepresentative": 0,
        "totalOpportunityDistricts": 0,
        "districts_summary": []


        }

        plan_specific_summary_data(plan_data, nv_plan_specific_data)
        smd_summary_data_plan_specific["interesting_plans_summary"].append(nv_plan_specific_data)

with open("Colorado/ensemble_data/co_smd_summary_data_plan_specific.json", 'w') as json_file:
    json.dump(smd_summary_data_plan_specific, json_file, indent=4)

        


#mmd 
mmd_plan_specific = ["Colorado/raw_data/CO_mmd_AVERAGE0.json",
                     "Colorado/raw_data/CO_mmd_DEMFAVORED0.json" ,
                     "Colorado/raw_data/CO_mmd_FAIR0.json", 
                     "Colorado/raw_data/CO_mmd_repfavored0.json"]
mmd_interesting_plans = ["AVERAGE","DEMFAVORED", "FAIR","REPFAVORED"]

mmd_interesting_plan_data = []

#load specific plan mmd data data
for plan_specific_file_path in mmd_plan_specific:
    with open(plan_specific_file_path, 'r') as file:
        mmd_plan_specific = json.load(file)
        mmd_interesting_plan_data.append(mmd_plan_specific)

mmd_summary_data_plan_specific = {
     "fips": "CO",
     "electionType": "MMD",
     "interesting_plans_summary": []

} 
for plan_name , plan_data in zip(mmd_interesting_plans, mmd_interesting_plan_data):
        nv_plan_specific_data = {
        "characteristic": plan_name,
        "partyWinner": None,
        "totalDistricts": 0,
        "totalPopulation": 0,
        "partyWinner": 0,
        "safeDistricts": 0,
        "totalRepresentative": 0,
        "totalOpportunityDistricts": 0,
        "districts_summary": []

        }

        plan_specific_summary_data(plan_data, nv_plan_specific_data)
        mmd_summary_data_plan_specific["interesting_plans_summary"].append(nv_plan_specific_data)

with open("Colorado/ensemble_data/co_mmd_summary_data_plan_specific.json", 'w') as json_file:
    json.dump(mmd_summary_data_plan_specific, json_file, indent=4)
 




In [216]:
def ensemble_smd_data_output(fips,shapefile,ensemble_data_paths,plan_specific_data_paths ,minority='black',opportunity_threshold=0.20, election_type='SMD'):
    races = ["white", "black", "asian", "hispanic", "other"]
    smd_interesting_plans = ["AVERAGE","DEMFAVORED", "FAIR","REPFAVORED", "ENACTED"]
    
    demographic_data = {
    "fips": fips,
    "electionType": election_type,
    "totalDistricts": 0,
    "barData": []
    }
    state_data = {
    "fips": "NV",
    "electionType": "SMD",
    "totalDistricts": 4,
    "democratAvgVoteShare": 0,
    "democratAvgSeatShare": 0,
    "republicanAvgVoteShare": 0,
    "republicanAvgSeatShare": 0,
    "barData": []

    }
    #load data 

    for ensemble_file_path in ensemble_data_paths:
        with open(ensemble_file_path, 'r') as file:
            smd_ensemble = json.load(file)
            collect_district_plan_demographic(smd_ensemble, shapefile ,demographic_data)
            election_result_collection(smd_ensemble, state_data)

    calculate_opportunity(demographic_data, state_data, 'black', 0.20)



    box_and_whisker_data = {
        "fips": "NV",
        "electionType": "SMD",
        "totalDistricts": 4,
        "boxes": []

        }

    collect_box_and_whisker_bins(demographic_data, box_and_whisker_data, races) 

    
    smd_interesting_plan_data = []

    #load specific plan data data
    for plan_specific_file_path in plan_specific_data_paths:
        with open(plan_specific_file_path, 'r') as file:
            smd_plan_specific = json.load(file)
            smd_interesting_plan_data.append(smd_plan_specific)

    for plan_name, plan_data in zip(smd_interesting_plans,smd_interesting_plan_data):
        plan_specific_data = {
            "fips": "NV",
            "electionType": "SMD",
            "totalDistricts": 4,
            "characteristic": plan_name,
            "demTotalVotes": 0,
            "repTotalVotes": 0,
            "districts": []
            }
        collec_metrics_plan_specific(plan_data, plan_specific_data, plan_name)
   


        plan_specific = {
            "fips": "NV",
            "electionType": "SMD",
            "totalDistricts": 4,
            "characteristic": plan_name,
            "democratsSeats": 0,
            "republicanSeats": 0,
            "demTotalVotes": 0,
            "repTotalVotes": 0,
            "bias": 0,
            "symmetry": 0,
            "responsiveness": 0,
            "curveData": []
            }
    vote_seat_share_data_collectiotion_specific_plan( plan_specific_data ,plan_specific, plan_name)
    calculate_annotation_for_vote_seat_share(nv_plan_specific)
    
    file_path = f'Nevada/smd_plan_specific/nv_smd_{plan_name}.json'
    with open(file_path, 'w') as json_file:
        json.dump(plan_specific, json_file, indent=4)



    return state_data, box_and_whisker_data




Index(['STATEFP', 'COUNTYFP', 'COUNTY', 'VTDST', 'NAME', 'G20PREDBID',
       'G20PRERTRU', 'G20PRELJOR', 'G20PREIBLA', 'G20PREONON', 'STATEFP20',
       'COUNTYFP20', 'VTDST20', 'GEOID20', 'NAME20', 'GEOID', 'ADJPOP',
       'TAWHITEALN', 'TABLACKCMB', 'TAAIANCMB', 'TAASIANCMB', 'TANHOPICMB',
       'TAOTHERALN', 'TA2RACE', 'TAHISPANIC', 'RACPOP_SUM', 'SCALING_FA',
       'WHT', 'AFAM', 'AIAN', 'ASN', 'HPAC', 'OTHR', 'LAT', 'geometry'],
      dtype='object')